#Library import

In [1]:
!nvidia-smi

Thu May 26 15:17:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/CP-Eng-3rd-year/Pattern Recog/Project/00 Library/')
!pip install --upgrade pythainlp
!pip install torchinfo
!pip install torchviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.5 MB 4.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4150 sha256=7a47ddd7c1eb1b0517fca018e4e8efcbd14c27654bff5fddaa6e79878abd0b19
  Stored in directory: /root/.cache/pip/wheels/04/38/f5/dc4f85c3909051823df49901e72015d2d750bd26b086480ec2
Successfully built torchviz


In [4]:
from data_processing_non import dictionary
from model_dataloader import EncoderDecoder

In [5]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm 
from torch.optim.lr_scheduler import ReduceLROnPlateau
import random


import torch
from torch import nn 
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torch.nn.functional as F
from torchvision import transforms, models
from torch.nn.utils.rnn import pad_sequence

from sklearn.model_selection import train_test_split

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

#Data loader


In [7]:
def get_identical_matrix(vector):
  identical_matrix = torch.eye(len(vector))

  for i in range(len(vector)):
    for j in range(len(vector)):
      identical_matrix[i, j] = vector[i] == vector[j]

  return identical_matrix

def get_n_gram_hadamard_operation(matrix, n = 4):
  outputs = []
  matrix_n = matrix
  outputs.append(matrix)

  w = matrix.shape[0]
  h = matrix.shape[1]

  for i in range(1, n):
    if w-i <= 0 or h-i <= 0:
      break
    matrix_n = matrix[:-i, :-i] * matrix_n[1:, 1:]
    outputs.append(matrix_n)

  return outputs 

def get_similarity_matrix(pred, caption):
  T = len(caption)
  output = torch.zeros(T, T)
  for i in range(T):
    for j in range(T):
      output[i, j] = pred[i, caption[j]].contiguous()
  return output

In [8]:
class FlickrDataset_Bleu_loss(Dataset):
    def __init__(self, 
                 captions_files, 
                 image_dir = "/content/drive/MyDrive/CP-Eng-3rd-year/Pattern Recog/Project/00 Data/archive/Images/", 
                 transform=None,
                 dictionary=dictionary,
                 test = False):
      
      self.image_dir = image_dir

      if type(captions_files) is str:
        self.dfs = pd.read_csv(captions_files)
        self.dfs = [self.df]
      elif type(captions_files) is list:
        for i in range(len(captions_files)):
          if  type(captions_files[i]) is str:
            captions_files[i] = pd.read_csv(captions_files[i])
        self.dfs = captions_files

      self.transform = transform
      self.test = test
      self.img_id = []
      self.img = {}
      self.caption = {}
      self.dictionary = dictionary

      self.caption_n_gram_count = {}

      unique_image = self.dfs[0]['image'].unique()

      for i in tqdm(range(len(unique_image))):
        img_id = unique_image[i]
        if img_id in self.caption:
          continue
        self.img_id.append(img_id)
        self.caption[img_id] = []
        self.caption_n_gram_count[img_id] = []
      
      for df in self.dfs:

        for i in tqdm(range(len(df))):
          img_id = df.iloc[i]['image']
          caption = df.iloc[i]['tokenized_caption_th']

          caption = caption[1:-1]
          caption = caption.split(",")
          caption = [word[1:].replace("'", "") for word in caption] 
          caption = self.dictionary.numericalize(caption)
          caption = torch.tensor(caption)
          
          identical_matrix = get_identical_matrix(caption[1:-1])
          n_gram_indentical_matrices = get_n_gram_hadamard_operation(identical_matrix)
          n_gram_count = [torch.sum(matrix, dim = 1) for matrix in n_gram_indentical_matrices]
          self.caption[img_id].append(caption)
          self.caption_n_gram_count[img_id].append(n_gram_count)
        
        
    def __len__(self):
      return len(self.img_id)
    
    def __getitem__(self, idx): 

      img_id = self.img_id[idx]
      if img_id not in self.img:
        img_path = os.path.join(self.image_dir, img_id)
        img = Image.open(img_path).convert("RGB")
        self.img[img_id] = img

      img = self.img[img_id]
      randomized_index = random.randint(0, len(self.caption[img_id])-1)
      caption = self.caption[img_id][randomized_index]
      caption_n_gram_count = self.caption_n_gram_count[img_id][randomized_index]

      if self.transform is not None:
        transformed_img = self.transform(img)

      if not self.test:
        return transformed_img, caption, caption_n_gram_count, img

      return transformed_img, caption, caption_n_gram_count, img, self.caption[img_id]
      

In [9]:
# Collate_fn for calling to transform a batch before return. 
class Collate_fn:
  def __init__(self, pad_value, batch_first=False):
    self.pad_value = pad_value       
    self.batch_first = batch_first    
        
  def __call__(self, batch):
    transformed_imgs = [item[0].unsqueeze(0) for item in batch]
    transformed_imgs = torch.cat(transformed_imgs, dim=0)    
        
    captions = [item[1] for item in batch]   
    captions = pad_sequence(captions, batch_first = self.batch_first, padding_value = self.pad_value)
    
    if len(batch[0]) > 4:
      return transformed_imgs, captions, [item[2] for item in batch], [item[3] for item in batch], [item[4] for item in batch]

    return transformed_imgs, captions, [item[2] for item in batch], [item[3] for item in batch]

In [10]:
dataframe_path = "/content/drive/MyDrive/CP-Eng-3rd-year/Pattern Recog/Project/00 Data/archive/"

In [11]:
train_captions_files = [dataframe_path+"captions_train.csv"]
val_captions_files = [dataframe_path+"captions_val.csv"]
test_captions_files = [dataframe_path+"captions_test.csv"]

image_size = 299

train_transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                          std=[0.5, 0.5, 0.5])
])

val_transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                          std=[0.5, 0.5, 0.5])
])

In [12]:
batch_size = 32
test_flickr_bleu_dataset = FlickrDataset_Bleu_loss(transform = val_transform, captions_files = test_captions_files, test = True)
test_flickr_bleu_dataloader = torch.utils.data.DataLoader( test_flickr_bleu_dataset, 
                                                batch_size=batch_size, 
                                                shuffle=True, 
                                                pin_memory=True, 
                                                collate_fn = Collate_fn(pad_value=0, batch_first = True)) 

  0%|          | 0/1214 [00:00<?, ?it/s]

  0%|          | 0/6070 [00:00<?, ?it/s]

#Model


In [13]:
#Config

embed_size=300
vocab_size = len(dictionary.stoi)
attention_dim=256
encoder_dim=2048
decoder_dim=512
learning_rate = 1e-3
number_of_decode_layer = 1

model_name = "model_1"
base_path = "/content/drive/MyDrive/CP-Eng-3rd-year/Pattern Recog/Project/01 Model/Anon/"
crossentropy_path = base_path + "baseline_"+model_name+"_state.pth"
mixed_path = base_path + model_name+"_state.pth"

In [14]:
model = EncoderDecoder(embed_size, vocab_size, attention_dim,encoder_dim,decoder_dim, number_of_decode_layer).to(device)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [15]:
def show_image_for_testing(all_captions, text_output, img):
  print()
  print("Predicted: ", "".join(text_output))
  for caption in all_captions:
    caption = [dictionary.itos[token.item()] for token in caption]
    reference.append(caption)
    print("Real: ", "".join(caption))
  print(sentence_bleu(reference, text_output))
  plt.imshow(img)
  plt.show()

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction, corpus_bleu
avg_bleu_score = 0
count = 0
max_count = 10
model.eval()

ref = list()
predict = list()

best = torch.load(crossentropy_path)
model.load_state_dict(best['state_dict'])


with torch.no_grad():
  for transformed_img, captions, caption_n_gram_count, img, all_captions in tqdm(test_flickr_bleu_dataloader):
    print(transformed_img.shape)
    transformed_img, captions = transformed_img.to(device), captions.to(device)
    text_output, _ = model.generate_text(transformed_img)
    for i in range(len(all_captions)):
      reference = []
      for caption in all_captions[i]:
        reference.append([dictionary.itos[token.item()] for token in caption if token not in [0, 1, 2, 3]])
      candidate = [ token for token in text_output[i] if dictionary.stoi[token] not in [0, 1, 2, 3]]
  
      ref.append(reference)
      predict.append(candidate)

      bleu = sentence_bleu(reference, candidate)
      if bleu <0.2:
        show_image_for_testing(all_captions[i], text_output[i], img[i])


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print("Avg TEST BLEU Score:", corpus_bleu(ref, predict))

#Caption


In [ ]:
img_id = ["3673165148_67f217064f.jpg",
"3611603026_9112b0c53f.jpg",
"2224995194_518859d97d.jpg",
"3264937930_9623496b64.jpg",
"1251558317_4ef844b775.jpg",
"3593556797_46b49a02a8.jpg",
"3421547427_53411b6278.jpg",
"3001612175_53567ffb58.jpg",
"3119887391_212f379797.jpg",
"3279025792_23bfd21bcc.jpg",
"3544233095_4bca71df1d.jpg",
"3462165890_c13ce13eff.jpg",
"2842032768_9d9ce04385.jpg",
"2909955251_4b326a46a7.jpg",
"3150742439_b8a352e1e0.jpg",
"2555622234_3e531e4014.jpg",
"519754987_51861fea85.jpg",
"3646453252_5ebbbaa6cc.jpg",
"513116697_ad0f4dc800.jpg",
"2942094037_f6b36fd3db.jpg",
"1.jpeg",
"2.jpeg",
"3.jpeg",
"4.jpeg",
"5.jpeg",
"6.jpeg",
"7.jpeg",
"8.jpeg",
"9.jpeg",
"10.jpeg"]

In [ ]:
class FlickrDataset_Caption(Dataset):
    def __init__(self, 
                 img_id,
                 image_dir = "/content/drive/MyDrive/CP-Eng-3rd-year/Pattern Recog/Project/00 Data/archive/Images/", 
                 transform=None):
      self.img_id = img_id
      self.img = {}
      self.image_dir = image_dir
      self.transform = transform
        
        
    def __len__(self):
      return len(self.img_id)
    
    def __getitem__(self, idx): 

      img_id = self.img_id[idx]
      if img_id not in self.img:
        img_path = os.path.join(self.image_dir, img_id)
        img = Image.open(img_path).convert("RGB")
        self.img[img_id] = img

      img = self.img[img_id]
      
      if self.transform is not None:
        transformed_img = self.transform(img)

      return transformed_img, img
      

In [ ]:
# Collate_fn for calling to transform a batch before return. 
class Collate_fn2:
  def __init__(self, ):
    pass
  def __call__(self, batch):
    transformed_imgs = [item[0].unsqueeze(0) for item in batch]
    transformed_imgs = torch.cat(transformed_imgs, dim=0)    
    return transformed_imgs, [item[1] for item in batch]

In [ ]:
caption_dataset = FlickrDataset_Caption(img_id, transform = val_transform)
caption_flickr_dataloader = torch.utils.data.DataLoader( caption_dataset, 
                                                batch_size=batch_size, 
                                                pin_memory=True,
                                                collate_fn = Collate_fn2()) 

In [ ]:
for transformed_img, img in caption_flickr_dataloader:
  transformed_img = transformed_img.to(device)
  text_output, _ = model.generate_text(transformed_img, max_length = 50)
  for i in range(len(text_output)):
    print("Predict", "".join(text_output[i]))
    plt.imshow(img[i])
    plt.show()
    print()

Output hidden; open in https://colab.research.google.com to view.